# Intro

- We need to complete 3 tasks
- Task1: divide question into multiple outcomes
- Task2: return a dictionary containing outcome as key and decision, p_yes, p_no, ... as value
- Task3: having the above values, return yes/no for the given question.

In [34]:
import crewai
import json
import os
from crewai import Agent
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
from pydantic import BaseModel
from crewai import Crew, Process, Agent, Task
search_tool = SerperDevTool()

In [39]:
class Outcomes(BaseModel):
    outcomes: list[str]

class ProbabilityOutput(BaseModel):
    decision: str
    p_yes: float
    p_no: float
    confidence: float

In [3]:
load_dotenv()

True

In [26]:
# Define your agents
researcher = Agent(
            role="Research Analyst",
            goal="Research and report on some future event, giving high quality and nuanced analysis",
            backstory="You are a senior research analyst who is adept at researching and reporting on future events.",
            verbose=True,
            allow_delegation=False,
            tools=[search_tool],
        )

analyst = Agent(
  role='Data Analyst',
  goal='Analyze research findings',
  backstory='A meticulous analyst with a knack for uncovering patterns'
)

writer = Agent(
  role='Writer',
  goal='Draft the final report',
  backstory='A skilled writer with a talent for crafting compelling narratives'
)

predictor = Agent(
            role="Professional Gambler",
            goal="Predict, based on some research you are presented with, whether or not a given event will occur",
            backstory="You are a professional gambler who is adept at predicting and betting on the outcomes of future events.",
            verbose=True,
            allow_delegation=False,
            tools=[]
        )

In [24]:
# ToDo - Test both research prompts
CREATE_OUTCOMES_FROM_SCENARIO_PROMPT =         """
        You are asked about a SCENARIO, which can have multiple, mutually-exclusive outcomes.
        You should break down the SCENARIO into a list of possible outcomes.
        
        Example 1:
        - SCENARIO: Will Gnosis Pay reach 100,000 users by 2025?
        - Answer: '''
            - Gnosis Pay reaches less than 25,000 users by 2025
            - Gnosis Pay reaches more than 25,000 users but less than 50,000 users by 2025
            - Gnosis Pay reaches more than 50,000 users but less than 75,000 users by 2025
            - Gnosis Pay reaches 100,000 users or more by 2025
        '''

        Example 2:
        - SCENARIO: Will the price of Bitcoin go up again in the next 2 months?
        - Answer: '''
            - The price of crypto will go up in the next 2 months
            - The price of crypto will not go up in the next 2 months
        '''
        
        [SCENARIO]
        {scenario}
        """
# research_prompt2 =         """
#         You are asked about a SCENARIO, which can have multiple, mutually-exclusive outcomes.
#         You should break down the SCENARIO into a list of possible outcomes.
        
#         [SCENARIO]
#         {scenario}
#         """

CREATE_OUTCOMES_FROM_SCENARIO_OUTPUT = '''
  A list containing multiple bullet points. Each bullet point should start with '-'.
  Each bullet point should contain a possible outcome resulting from the
  provided SCENARIO. The produced outcomes should be mutually exclusive, i.e. only one of them should be true whereas
  the remaining ones should be false.
  '''

In [29]:
def print_outcomes(question:str) -> None:
    create_outcomes_task = Task(
          description=(CREATE_OUTCOMES_FROM_SCENARIO_PROMPT),
          expected_output=CREATE_OUTCOMES_FROM_SCENARIO_OUTPUT,
          #tools=[search_tool],
          tools = [],
          output_json=Outcomes,
          agent=researcher,
        )

    report_crew = Crew(
          agents=[researcher],
          tasks=[create_outcomes_task],
    )
    result = report_crew.kickoff(inputs={'scenario': question})
    print(result)
    return result

In [49]:
question = 'Will smart accounts replace EOAs in the EVM ecosystem in the next 3-5 years?'
result = print_outcomes(question)

2024-04-03 18:11:11,015 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Since it is a future event, I need to research the current trends and opinions around the use of smart accounts in the EVM ecosystem. This will help me identify the possible outcomes for the next 3-5 years. 

Action: 
Search the internet

Action Input: 
{
"search_query": "future of smart accounts in EVM ecosystem"
} 


Search results: Title: Ethereum devs debate future of account abstraction - Blockworks
Link: https://blockworks.co/news/ethereum-devs-debate-account-abstraction
Snippet: Ethereum devs debate future of account abstraction ... EVM ecosystem towards smart contract accounts (SCAs). ... future trajectory of the network.
---
Title: State of Wallets - Part 2: Smart Accounts (Account Abstraction
Link: https://www.coinbase.com/blog/state-of-wallets-2
Snippet: Account Abstraction (“AA”) launched in the EVM ecosystem in Q1-2023 and is beginning to see an uptick in adoption.
---
Title: Future of Smart Accounts: Modular, Specialised & Multi

In [36]:
json.loads(result)['outcomes']

['Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years',
 'Smart accounts will become the dominant form of accounts, but EOAs will still be used in the EVM ecosystem within the next 3-5 years',
 'Both smart accounts and EOAs will continue to coexist with no dominant form in the EVM ecosystem within the next 3-5 years',
 'EOAs will remain the dominant form of accounts while smart accounts adoption grows in the EVM ecosystem within the next 3-5 years',
 'EOAs will continue to be the dominant form of accounts, and smart accounts will not gain significant traction in the EVM ecosystem within the next 3-5 years']

In [40]:
# ToDo - Test both research prompts
RESEARCH_OUTCOME_PROMPT = """
                Research and report on the following sentence:
                {sentence}
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                """
RESEARCH_OUTCOME_OUTPUT = """
            Return your answer in raw JSON format, with no special formatting such as newlines, as follows:
                {{"report": <REPORT>}}
                where <REPORT> is a free text field that contains a well though out justification 
                for the predicition based on the summary of your findings.
            """

In [55]:

def generate_prediction_for_one_outcome(sentence: str):

    task_research_one_outcome = Task(
                description=(RESEARCH_OUTCOME_PROMPT),
                tools=[search_tool],
                agent=researcher,
                expected_output=(RESEARCH_OUTCOME_OUTPUT),
            )
    task_create_probability_for_one_outcome = Task(
                description=("""
                        Your task is to determine the probability of a prediction market affirmation being answered 'Yes' or 'No'.
    Use the sentence provided in 'SENTENCE' and follow these guidelines:
    * Focus on the affirmation inside double quotes in 'SENTENCE'.
    * The question must have only 'Yes' or 'No' outcomes. If not, respond with "Error".
    * Use the tools provided to aid your estimation.
    * Evaluate recent information more heavily than older information.
    * Your response should include:
        - "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
        - "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
        - "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
        - "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 1 (maximum confidence). 
        Confidence can be calculated based on the quality and quantity of data used for the estimation.
        
        Ensure p_yes + p_no equals 1.
    
        SENTENCE: {sentence}
                  """
                ),
            expected_output="""
        Your response should include:
    - "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
    - "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 
    1 (maximum confidence). Confidence can be calculated based on the quality and quantity of data used for the estimation.
    
    Format your response in JSON format, including the keys "decision", "p_yes", "p_no" and "confidence".
            """,
                agent=predictor,
            output_json=ProbabilityOutput,
        context=[task_research_one_outcome]
            )
    crew = Crew(
                agents=[researcher, predictor],
                tasks=[task_research_one_outcome, task_create_probability_for_one_outcome],
                verbose=2,
            process=Process.sequential            
            )
    
    #sentence = 'The temperature will be above 20C on May 24, 2024 in Berlin.'
    sentence = 'Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years'
    result = crew.kickoff(inputs={'sentence': sentence})
    return result

In [56]:
#sentence = 'The temperature will be above 20C on May 24, 2024 in Berlin.'
sentence = 'Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years'
result = generate_prediction_for_one_outcome(sentence)

2024-04-03 18:14:18,407 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
I need to research whether smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years. This requires an understanding of what smart accounts and EOAs are, how they function in the EVM ecosystem currently, and any potential developments that could lead to smart accounts replacing EOAs. To gather this information, I'll need to search the internet.

Action: Search the internet
Action Input: {"search_query": "Smart accounts vs EOAs in the EVM ecosystem"} 


Search results: Title: EOA vs. Smart Contract Account: What Is the Difference?
Link: h

In [60]:
prediction_for_outcome = ProbabilityOutput.parse_obj(json.loads(result))

In [59]:
outcomes

Outcomes(outcomes=['Smart accounts will become the dominant type of account in the EVM ecosystem, replacing EOAs in most use cases', 'Smart accounts will coexist with EOAs, with each type of account being used for different applications', 'Smart accounts will only be used in specialized applications, while EOAs will continue to be the most common type of account', 'Smart accounts will not gain further adoption and EOAs will remain the dominant type of account'])

In [61]:
prediction_for_outcome

ProbabilityOutput(decision='n', p_yes=0.4, p_no=0.6, confidence=0.7)

In [71]:
outcomes_with_probs = []
for outcome in outcomes.outcomes:
    prediction = generate_prediction_for_one_outcome(outcome)
    probability_output = ProbabilityOutput.parse_raw(prediction)
    outcomes_with_probs.append((outcome, probability_output))

2024-04-03 18:27:46,758 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
I first need to understand the context and the key terms in the sentence. Specifically, I need to understand the meaning of "Smart accounts", "EOAs", and "EVM ecosystem". I will then research the current trends and predictions about the future of these terms in the next 3-5 years. 

Action: 
Search the internet 

Action Input: 
{"search_query": "What are Smart accounts, EOAs, and EVM ecosystem"} 


Search results: Title: EOA vs. Smart Contract Account: What Is the Difference?
Link: https://blog.ambire.com/eoas-vs-smart-contract-accounts/
Snippet: EOAs and Smart Con

2024-04-03 18:28:35,141 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Professional Gambler] Task output: {
  "decision": "n",
  "p_yes": 0.2,
  "p_no": 0.8,
  "confidence": 0.85
}


 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
I need to understand more about the terms "Smart accounts", "EOAs", and "EVM ecosystem" in order to answer this question effectively. I will start my research by searching for these terms on the internet. 

Action: 
Search the internet

Action Input: 
{"search_query": "What are Smart accounts in EVM ecosystem"} 


Search results: Title: What are Smart Accounts on Ethereum? - Coinspeaker
Link: https://www.coinspeaker.com/guides/what-are-smart

2024-04-03 18:29:24,860 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Professional Gambler] Task output: {
  "decision": "n",
  "p_yes": 0.4,
  "p_no": 0.6,
  "confidence": 0.7
}


 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
To give a well-rounded analysis, I need to understand what Smart Accounts and EOAs (Externally Owned Accounts) are, and their current state in the EVM (Ethereum Virtual Machine) ecosystem. I also need to find any trends, predictions, or expert opinions on their future use and advancements within the next 3-5 years. I will begin by researching these topics.

Action: 
Search the internet

Action Input: 
{"search_query": "Smart accounts vs EOAs 

2024-04-03 18:30:16,658 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == [Professional Gambler] Task output: {
  "decision": "y",
  "p_yes": 0.7,
  "p_no": 0.3,
  "confidence": 0.6
}


 [DEBUG]: == Working Agent: Research Analyst
 [INFO]: == Starting Task: 
                Research and report on the following sentence:
                Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years
                Search and scrape the web for information that will help you give a high quality, nuanced answer to the question.
                


> Entering new CrewAgentExecutor chain...
First, I need to understand the key terms in the task. Smart Accounts and EOAs (Externally Owned Accounts) are terms related to Ethereum's blockchain technology. EVM refers to Ethereum Virtual Machine. I need to find out what smart accounts are, how they compare to EOAs, and what trends or changes are expected in the next 3-5 years regarding their usage in the EVM ecosystem. I'll start by researching each of these key terms individually, then

In [72]:
outcomes_with_probs

[('Smart accounts will become the dominant type of account in the EVM ecosystem, replacing EOAs in most use cases',
  ProbabilityOutput(decision='n', p_yes=0.2, p_no=0.8, confidence=0.85)),
 ('Smart accounts will coexist with EOAs, with each type of account being used for different applications',
  ProbabilityOutput(decision='n', p_yes=0.4, p_no=0.6, confidence=0.7)),
 ('Smart accounts will only be used in specialized applications, while EOAs will continue to be the most common type of account',
  ProbabilityOutput(decision='y', p_yes=0.7, p_no=0.3, confidence=0.6)),
 ('Smart accounts will not gain further adoption and EOAs will remain the dominant type of account',
  ProbabilityOutput(decision='y', p_yes=0.6, p_no=0.4, confidence=0.7))]

In [73]:
# Finally, final LLM that makes a decision

In [92]:
FINAL_DECISION_PROMPT = """
Your task is to determine the probability of a given outcome being true.
Below you can find the , a list of tuples where the first element
is an outcome

[Outcomes with probabilities]
"""

FINAL_DECISION_PROMPT = """
Your task is to determine the probability of a binary outcome being answered 'Yes' or 'No'.
OUTCOMES_WITH_PROBABILITIES is provided, which contains a set of {number_of_outcomes} outcomes.
The object OUTCOMES_WITH_PROBABILITIES contains a list of tuple, where each tuple contains, as its
first element, an outcome, and as its second entry, a probability dictionary, having properties such
 as p_yes (probability that the outcome is true), p_no (probability that the outcome is false),
confidence (confidence level in the values of p_yes and p_no).
Observe that the outcomes contained inside OUTCOMES_WITH_PROBABILITIES are mutually exclusive, i.e.
only one of them can be true. 
You should determine the probability of the outcome OUTCOME_TO_ASSESS being true, 
considering the probabilities of the other related outcomes.

OUTCOME_TO_ASSESS: {outcome_to_assess}
OUTCOMES_WITH_PROBABILITIES: {outcomes_with_probabilities}
"""                
FINAL_DECISION_OUTPUT="""
        Your response should include:
    - "decision": The decision you made. Either `y` (for `Yes`) or `n` (for `No`).
    - "p_yes": Probability that the sentence outcome will be `Yes`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "p_no": Probability that the sentence outcome will be `No`. Ranging from 0 (lowest probability) to 1 (maximum probability).
    - "confidence": Indicating the confidence in the estimated probabilities you provided ranging from 0 (lowest confidence) to 
    1 (maximum confidence). Confidence can be calculated based on the quality and quantity of data used for the estimation.

  Ensure p_yes + p_no equals 1.
    
    Format your response in JSON format, including the keys "decision", "p_yes", "p_no" and "confidence".
"""

In [133]:
def generate_final_decision(outcomes_with_probabilities):

    task_final_decision = Task(
                description=(FINAL_DECISION_PROMPT),
                tools=[],
                agent=predictor,
                expected_output=(FINAL_DECISION_OUTPUT),
            output_json=ProbabilityOutput,
            )
    crew = Crew(
                agents=[predictor],
                tasks=[task_final_decision],
                verbose=2,
            process=Process.sequential            
            )
    
    sentence = 'Smart accounts will fully replace EOAs in the EVM ecosystem within the next 3-5 years'
    result = crew.kickoff(inputs={'outcomes_with_probabilities': 
                                  [(i[0],i[1].dict()) for i in outcomes_with_probabilities],
                                 'number_of_outcomes': len(outcomes_with_probabilities),
                                 'outcome_to_assess': outcomes_with_probabilities[0][0]})
    return result

In [134]:
# Run LLM to get these values again
b = [('Smart accounts will become the dominant type of account in the EVM ecosystem, replacing EOAs in most use cases',
  ProbabilityOutput(decision='n', p_yes=0.2, p_no=0.8, confidence=0.85)),
 ('Smart accounts will coexist with EOAs, with each type of account being used for different applications',
  ProbabilityOutput(decision='n', p_yes=0.4, p_no=0.6, confidence=0.7)),
 ('Smart accounts will only be used in specialized applications, while EOAs will continue to be the most common type of account',
  ProbabilityOutput(decision='y', p_yes=0.7, p_no=0.3, confidence=0.6)),
 ('Smart accounts will not gain further adoption and EOAs will remain the dominant type of account',
  ProbabilityOutput(decision='y', p_yes=0.6, p_no=0.4, confidence=0.7))]

In [135]:
final_decision = generate_final_decision(b)

2024-04-03 19:15:41,257 - 136084017362176 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Professional Gambler
 [INFO]: == Starting Task: 
Your task is to determine the probability of a binary outcome being answered 'Yes' or 'No'.
OUTCOMES_WITH_PROBABILITIES is provided, which contains a set of 4 outcomes.
The object OUTCOMES_WITH_PROBABILITIES contains a list of tuple, where each tuple contains, as its
first element, an outcome, and as its second entry, a probability dictionary, having properties such
 as p_yes (probability that the outcome is true), p_no (probability that the outcome is false),
confidence (confidence level in the values of p_yes and p_no).
Observe that the outcomes contained inside OUTCOMES_WITH_PROBABILITIES are mutually exclusive, i.e.
only one of them can be true. 
You should determine the probability of the outcome OUTCOME_TO_ASSESS being true, 
considering the probabilities of the other related outcomes.

OUTCOME_TO_ASSESS: Smart accounts will become the dominant type of account in the EVM ecosystem, replacing EOAs in most

In [137]:
ProbabilityOutput.parse_raw(result)

ProbabilityOutput(decision='n', p_yes=0.4, p_no=0.6, confidence=0.7)

In [141]:
ProbabilityOutput.model_validate_json(result)

ProbabilityOutput(decision='n', p_yes=0.4, p_no=0.6, confidence=0.7)